# Replicating the unit cell of a Silicene Lattice

In [1]:
import numpy as np
import inspect
import os

### General File Path Definition

Defines the file location to the directory holding the iPython notebook.

In [2]:
file_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
file_name = 'POSCAR_ZrB2_silicene_11x2_v2.vesta'
file_path = os.path.join(file_dir, file_name)

### File Read Test

In [3]:
file = open(file_path, 'r')
print(file.readline())
file.close()

#VESTA_FORMAT_VERSION 3.1.9



### Read File

Reads in file as a single string.

In [15]:
file = open(file_path, 'r')
file_data = file.read()
file.close()

Split string by newline '\n' commands.

In [18]:
file_data = file_data.split('\n')

Find the index numbers of the CELLP and STRUC keywords.

In [20]:
cellp_indx = file_data.index('CELLP')
struc_indx = file_data.index('STRUC')

### Get CELLP data

In [42]:
cellp_data = []
for i in range(cellp_indx, struc_indx):
    cellp_data.append(file_data[i])
for i in range(1,len(cellp_data)):
    cellp_data[i] = np.fromstring(cellp_data[i], dtype=float, sep='  ')

In [43]:
cellp_data

['CELLP',
 array([ 60.426811,   6.34317 ,  26.60656 ,  90.      ,  90.      ,  90.      ]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.])]

### STRUC Data

Defines index of next keyword after STRUC.

In [45]:
theri_indx = file_data.index('THERI 0')

Separates out the structure data from file_data.

In [76]:
struc_data = []
for i in range(struc_indx, theri_indx):
    struc_data.append(file_data[i])

Line wise splits single string by white space and then removes elements that are empty.

In [77]:
for i in range(1, len(struc_data)):
    temp_line = np.array(struc_data[i].split(' '))
    temp_line_1 = np.array([])
    for j in range(0, len(temp_line)):
        if not temp_line[j] == '':
            temp_line_1 = np.append(temp_line_1, temp_line[j])
    struc_data[i] = temp_line_1
del temp_line, temp_line_1    

A function to check if a string can be converted to a float.

In [104]:
def str2float_check(string):
    try:
        float(string)
        return True
    except ValueError:
        return False

iterates over the structure data linewise, and then element wise to convert strings that contain a number to a float.

In [117]:
for i in range(1, len(struc_data)):  # iterates over lines of struc_data
    temp_line = []  # create a temporary line
    for j in range(len(struc_data[i])):  # iterates over elements of line 'i' in struc_data
        if str2float_check(struc_data[i][j]):  # returns true if element can be converted to a float.
            temp_line.append(float(struc_data[i][j]))
        else:
            temp_line.append(struc_data[i][j])
    struc_data[i] = temp_line
del temp_line

Below, I open our vesta file in reading and writing mode.
Then, I extract the cellparameters (CELLP) from the file, and save them as a numpy array.
This will be used later to iterate over the structure. CELLPL is a numpy array of just the cell parameters, a,b &c.

In [ ]:
f = open("POSCAR_silicene_ZrB2_Tedit.vesta", "r") #opens the file in reading mode
for line in f:
    if "CELLP" in line:
        for line in f:
            if "60.426811" in line:
                CELLP = np.fromstring(line, dtype=float, count=-1, sep="   ") #creates CELLP as a 1D numpy array
                break
            else:
                break
print(CELLP)
CELLPL = CELLP[0:3]
print(CELLPL)

The cell below reads the "STRUC" section line by line, saving each line as a line in the list STRUC.
The line below it tests this, by printing STRUC[0]. Later, these lines will be converted into numpy arrays, which can be operated on.

In [ ]:
f = open("POSCAR_silicene_ZrB2_Tedit.vesta", "r+")
STRUC = []
for line in f:
    if "STRUC" in line:
        STRUC.append(f.readline())
        f.readline()
        STRUC.append(f.readline())
f.close()

In [ ]:
STRUC[0]

The code below splits the element STRUC[0], which is one big  string, by its whitespace. Later, we will remove the whitespace.

In [ ]:
STRUC[0] = STRUC[0].split(" ")
STRUC[0]

The code below removes the whitespace in STRUC[0]. It has to be iterated several times, because for some reason the 2nd-4th lines don't remove all of the whitespace first time.

In [ ]:
for number in range(len(STRUC[0])):
    for item in STRUC[0]:
        if item == '':
            STRUC[0].remove(item)
print(STRUC[0])

***

#### Suggested Edit

Thomas I think what is occuring is that since you are removing items from STRUC[0] there is a 'confusion' about the number of elements to iterate over and what they point to.
You may want to try something like this.

I needed to create test_list as I didn't have the VESTA file.

In [ ]:
test_list = ['','', '1', 'Zr', '', '', '', '', '', '', '', 'Zr1', '', '1.0000', '', '', '0.000000', '', '', '0.000000', '', '',
             '0.066667', '', '', '', '1a', '', '', '', '', '', '', '1\n']

Lets create a new temp list and check to see if each element of test_list is white space. If it is we will not add it to temp_struc. However, if it isn't white space we will add it to temp_struc.
At the end we will do some redefining to clean things up a bit.

In [ ]:
temp_struc = []
for item in test_list:
    if not item == '':
        temp_struc.append(item)
test_list = temp_struc
del temp_struc

Lets look at test_list again and see if we have what we want.

In [ ]:
test_list

Looks like it has worked!

***

Below, we convert all of the numbers in STRUC[0] into floats - the strings like "Zr" remain as strings. 

In [ ]:
alphabet = ( "B", "Zr", "Si", "1a") #can be generalised further
temp_list = []
for item in STRUC[0]:
    if not any(s in item for s in alphabet):
        temp_list.append(float(item))
    else:
        temp_list.append(item)
STRUC[0] = temp_list
del temp_list
print(STRUC[0])

The code below converts elements 4, 5 & 6 in the list STRUC[0] into a numpy array.

In [ ]:

STRUC[0].insert(4, np.array(STRUC[0][4:7]))
del STRUC[0][5:8]
print(STRUC[0])

Below I will add our CELLP to STRUC[0][4] and assess the result. CELLPL is a 3 element numpy array with only the unit cell length parameters. STRUC[0][4] is a numpy array of elements 4, 5 & 6 in STRUC[0]. 

In [ ]:
test = STRUC[0][:]
print(test)
test[4] = test[4] + CELLPL
print(test)
print(STRUC[0])

Now, "test" should be our modified STRUC[0], moved along each axis by 1 cell parameter. Our next step will be to write a new file with the data from test, and check if it can be read in vesta.

In [ ]:
test_1 = test[0:4]
test_2 = test[5:]
test_3 = test[4].tolist()
test = test_1 + test_3 + test_2
print(test)

In [ ]:
f = open("testfile.txt", "w")
f.write("#VESTA_FORMAT_VERSION 3.3.0")
f.write("CRYSTAL")
f.close()

***

### Try to keep the file tidy and get rid of redundant testing cells. 

In [ ]:
a=1

In [ ]:
f.close()

In [ ]:
test1 = np.array([])

In [ ]:
a